**IMAGE PREPROCESSING**

**IMPORT THE NECESSARY LIBRARIES**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from google.colab.patches import cv2_imshow

from skimage import feature 
from imutils import build_montages 
from imutils import paths

import numpy as np
import cv2

import os 
import pickle


PATH FOR TRAIN AND TEST DATA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
trainingpath=r"/content/drive/My Drive/dataset/spiral/training"  
testingpath=r"/content/drive/My Drive/dataset/spiral/testing"

QUANTIFYING IMAGES

In [ ]:
def quantify_image(image):
    features = feature.hog(image, orientations=9,
                           pixels_per_cell=(10, 10), 
                           cells_per_block=(2, 2),
                           transform_sqrt=True, 
                           block_norm="L1")
    return features

LOADING TRAIN DATA AND TEST DATA

In [ ]:
def load_split(path):
  #grab the list of img in the input directory, the initialize the list of data images and class labels
  imagePaths = list(paths.list_images (path))
  data=[]
  labels=[]
  #loop over the image paths 
  for imagePath in imagePaths:
    #extract the class label from the filename 
    label = imagePath.split(os.path.sep) [-2]
    #load the input image, convert it to grayscale, and resize # it to 200x200 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (200, 200))
    #threshold the image such that the drawing appears as white on a black background
    image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV | cv2. THRESH_OTSU) [1]
    features = quantify_image(image) # quantify the image 
    data.append(features)
    labels.append(label)#update the data and labels lists, respectively 
  return (np.array(data), np.array(labels))

In [ ]:
print("[INFO] loading data...")
(X_train, y_train) = load_split(trainingpath)
(X_test, y_test) = load_split(testingpath)


[INFO] loading data...


LABEL ENCODING

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
print(X_train.shape,y_train.shape)


(72, 12996) (72,)


**MODEL BUILDING**

TRAINING THE MODEL

In [ ]:
print("[INFO] training model")
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

[INFO] training model


RandomForestClassifier()

TESTING THE MODEL

In [ ]:
testingpath=list(paths.list_images(testingpath))
idxs=np.arange(0,len(testingpath))
idxs=np.random.choice(idxs,size=(25,),replace=False)
images=[]

In [ ]:
from google.colab.patches import cv2_imshow
for i in idxs:
    image=cv2.imread(testingpath[i])
    output=image.copy()
        
    # load the input image,convert to grayscale and resize
    
    output=cv2.resize(output,(128,128))
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image=cv2.resize(image,(200,200))
    image=cv2.threshold(image,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    #quantify the image and make predictions based on the  extracted feature using last trained random forest
    features=quantify_image(image)
    preds=model.predict([features])
    label=le.inverse_transform(preds)[0]
    #the set of output images
    if label=="healthy":
        color=(0,255,0)
    else:
        color=(0,0,255)
        
    cv2.putText(output,label,(3,20),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)
    images.append(output)

#creating a montage
montage=build_montages(images,(128,128),(5,5))[0]
cv2.waitKey(0)

-1

MODEL EVALUATION

In [ ]:
predictions = model.predict(X_test)

cm = confusion_matrix(y_test, predictions).flatten()
print(cm)
(tn, fp, fn, tp) = cm
accuracy = (tp + tn) / float(cm.sum())
print(accuracy)

[14  1  3 12]
0.8666666666666667


SAVE THE MODEL

In [ ]:
pickle.dump(model,open('parkinson.pkl','wb')) 

DEPLOYMENT

In [ ]:
!pip install -U ibm-watson-machine-learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.8 MB/s 
     |████████████████████████████████| 51 kB 702 kB/s 
     |████████████████████████████████| 824 kB 62.2 MB/s 
     |████████████████████████████████| 133 kB 51.2 MB/s 
     |████████████████████████████████| 547 kB 44.3 MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.7.0-py2.py3-none-any.whl size=72563 sha256=0c73fd5ff2723ac7302b1d38719d5a86bfd888f4989ca195b98253a242f80f19
  Stored in directory: /root/.cache/pip/wheels/47/22/bf/e1154ff0f5de93cc477acd0ca69abfbb8b799c5b28a66b44c2
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.7.0-py2.py3-none-any.whl size=501013 sha256=e1fc8c4f5fde9962b0c685463c122d0baf41a331a64e948f24912c611a91b147
  Stored in directory: /root/.cache/pip/wheels/6c/a2/e4/c16d02f809a3ea998e17cfd02c13369281f3d232aaf5902c19
  Created wheel for ibm-cos-sdk-s3transfer: filename=ibm_cos_

In [ ]:
# Now connect notebook ml service with api key and url

from ibm_watson_machine_learning import APIClient
import json
import numpy as np

AUTENTICATE AND SET SPACE

In [ ]:
wml_credentials = {
    "apikey" : "RYa2JTvIsfgzBUbvFxnCYVUxLBDntmTWzc9KGStjRtC5",
    "url" : "https://us-south.ml.cloud.ibm.com" #For Dallas region
}

In [ ]:
wml_client =APIClient(wml_credentials)

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.


In [ ]:
# Check the available deployments

wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -------------------------  ------------------------
ID                                    NAME                       CREATED
efa48345-def9-4aa5-b19f-4dd7d5f766ce  ParkinsonDiseaseDetection  2022-11-06T10:09:49.894Z
------------------------------------  -------------------------  ------------------------


In [ ]:
SPACE_ID = "efa48345-def9-4aa5-b19f-4dd7d5f766ce"


In [ ]:
# Space id created default one

wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [ ]:
# To check the environment

wml_client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

SAVE AND DEPLOY THE MODEL


In [ ]:
import sklearn
sklearn.__version__

'1.0.2'

In [ ]:
MODEL_NAME = "ParkinsonDiseaseDetection_DeployedModel"
DEPLOYMENT_NAME = "ParkinsonDiseaseDetection"
DEMO_MODEL = Models.model

In [ ]:
# Set Python default version

software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

Create Model Properties to deploy the model

In [ ]:
# Setup Model Meta

model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

In [ ]:
# Save Model

model_details = wml_client.repository.store_model{
    model = DEMO_MODEL,
    meta_props = model_props,
    training_data = X_train,
    training_target = y_train
}

In [ ]:
model_details

In [ ]:
model_id = wml_client.repository.get_model_id(model_details)
model_id

In [ ]:
# Download the model locally

wml_client.repository.download('9a0b7baa-7377-4117-a020-9e5d7b26974d','Deployed_Model.tar.gz')

Successfully saved model content to file: 'Deployed_Model.tar.gz'


'/content/Deployed_Model.tar.gz'

DEPLOY IN PROPS


In [ ]:
# Set meta

deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME : DEPLOYMENT_NAME,
    wml_client.deployments.ConfigurationMetaNames.ONLINE : {}
}

In [ ]:
# Deploy

deployment = wml_client.deployments.create(
    artifact_uid = model_id
    meta_props = deployment_props
)